In [2]:
import numpy as np
import pandas as pd
import time
import math
from scipy import signal
import read_ppg


from datetime import datetime
import csv

In [25]:
def stage_value(x):
    if x == 'REM':
        return 12
    elif x == 'N1':
        return 13
    elif x == 'N2':
        return 14
    elif x == 'N3':
        return 15
    else:
        return 11

def trans(x):
   
    with open(x) as f:
        a = []
        SleepStage = []
        data = f.readlines()  

        for line in data:
            s = line.split('\t')
            SleepStage.append(s[0])

        SleepStage1 = SleepStage[SleepStage.index("Sleep Stage") + 1:]

        for i in SleepStage1:
            STAGE = stage_value(i)
            a.append(STAGE)
            
        t = data[SleepStage.index("Sleep Stage") + 1].split('\t')[2]
        
        st = datetime.strptime(t[:19], '%Y-%m-%dT%H:%M:%S')# 加入開始時間
        start_in_sec = (st.hour*60 + st.minute)*60 + st.second
    return a, start_in_sec ,t


file = 'data/20220621_ike_raw/1110621-stage.txt'

STAGE = trans(file)[0]
start_in_sec= trans(file)[1]

f = open('220621_Tai_STAGE.csv', 'w')
writer = csv.writer(f)
writer.writerow(STAGE)
f.close()


In [4]:
def read_raw(x):

    data = [[] for i in range(6)]
    # represent_data_r = []
    # represent_data_ir = []
    # represent_data_g = []
    ppg_data = []
    time = []

    imu_freq = 1  # Hz
    imu_start = 14
    imu_data_size = 2  # byte
    imu_type = 3
    imu_period = imu_data_size*imu_type  # xyz
    ppg_freq = 10  # Hz
    ppg_start = imu_start+imu_data_size*3 + (imu_freq-1)*imu_period  # is empty
    ppg_data_size = 4  # byte
    ppg_type = 4
    ppg_period = ppg_data_size*ppg_type  # g/r/ir/e

    rows = x.split('\n')

    for line in rows:
        lines = line
        if line == "":
            continue
        line = bytearray.fromhex(''.join(line.split(' ')[:-1]))
        for i in range(ppg_freq):
            for j in range(ppg_type):
                start = ppg_start+j*ppg_data_size
                end = ppg_start+(j+1)*ppg_data_size
                ppg_data.append(int(int.from_bytes(
                    line[start+i*ppg_period:end+i*ppg_period], byteorder="little")))
            data[0].append(ppg_data[0])
            data[1].append(ppg_data[1])
            data[2].append(ppg_data[2])
            c = lines.split(' ')
            time.append(c[-1:])
    time = [b for a in time for b in a]
    R = np.array(data[0])
    IR = np.array(data[1])
    G = np.array(data[2])
    T = np.array(time)

    df = pd.DataFrame({'R': R,
                       'IR': IR,
                       'G': G,
                       'T': T})
    df.to_csv('220621_Tai_raw.csv', index=False, header=False)


f = open('data/20220621_ike_raw/ppg_raw.txt')
text = f.read()
read_raw(text)


In [51]:
# Read the apnea events (2 Hz)
file = 'data/20220621_Ike_raw/1110621-Events.txt'

f=open('data/20220621_ike_raw/1110621-stage.txt')

SleepStag = []
date =[]

data = f.readlines()  

for line in data:
    s = line.split('\t')
    SleepStag.append(s[0])
    
print(SleepStag.index("Sleep Stage") + 1)
for line in data[SleepStag.index("Sleep Stage") + 1:]:
    t = line.split('\t')[2]
    
    date.append(t[:19])
    
with open(file) as file_apnea :
    data = file_apnea.readlines() 
    
    CSA = 1
    OSA = 2
    MIX = 2
    HYP = 3
    Sleepevent =[]
    SleepStage =[]
    Duration=[]
    
    total_sec=23947
    apnea = np.zeros(total_sec*2)
    
    for line in data:
            s = line.split('\t')
            SleepStage.append(s[0])

    for line in data[SleepStage.index("Sleep Stage") + 1:]:
        e = line.split('\t')
        
        Sleepevent.append(e[3])
        Duration.append(e[4])
    
    for i in range(len(Sleepevent)):
        
        x = Sleepevent[i]
        tm = datetime.strptime(date[i][:19], '%Y-%m-%dT%H:%M:%S')
        #print(tm)
        hh = tm.hour
        if hh<12:
            hh = hh+24
        mm = tm.minute
        ss = tm.second
        
        if x=='Central apnea' or x=='Hypopnea Central':
            dura = int(2* float(Duration[i]))
            Time_in_sec = (hh*60 + mm)*60 + ss
            if (Time_in_sec - start_in_sec) < 0:
                print('Error time.')
                print(severity + ' - ' + key + ' at row ' + str(i+1))
            event_start = max(0, Time_in_sec - start_in_sec) * 2
            event_end   = max(0, ((Time_in_sec - start_in_sec) * 2 + dura))
            apnea[event_start : event_end] = CSA
            
        elif x=='Obstructive apnea' or x=='Hypopnea Obstructive':
            dura = int(2* float(Duration[i]))
            Time_in_sec = (hh*60 + mm)*60 + ss
            if (Time_in_sec - start_in_sec) < 0:
                print('Error time.')
                print(severity + ' - ' + key + ' at row ' + str(i+1))
            event_start = max(0, Time_in_sec - start_in_sec) * 2
            event_end   = max(0, ((Time_in_sec - start_in_sec) * 2 + dura))
            apnea[event_start : event_end] = OSA
            
        elif x=='Mixed apnea' or x=='Hypopnea Mixed':
            dura = int(2* float(Duration[i]))
            Time_in_sec = (hh*60 + mm)*60 + ss
            if (Time_in_sec - start_in_sec) < 0:
                print('Error time.')
                print(severity + ' - ' + key + ' at row ' + str(i+1))
            event_start = max(0, Time_in_sec - start_in_sec) * 2
            event_end   = max(0, ((Time_in_sec - start_in_sec) * 2 + dura))
            apnea[event_start : event_end] = MIX
            
        elif x=='Hypopnea':
            dura = int(2* float(Duration[i]))
            print(dura)
            Time_in_sec = (hh*60 + mm)*60 + ss
            print(Time_in_sec)
            if (Time_in_sec - start_in_sec) < 0:
                print('Error time.')
                print(severity + ' - ' + key + ' at row ' + str(i+1))
            event_start = max(0, Time_in_sec - start_in_sec) * 2
            event_end   = max(0, ((Time_in_sec - start_in_sec) * 2 + dura))
            apnea[event_start : event_end] = HYP

        
f = open('220621_Tai_apnea.csv', 'w')
writer = csv.writer(f)
writer.writerow(apnea)
f.close()

17
['2022-06-21T22:12:41', '2022-06-21T22:13:11', '2022-06-21T22:13:41', '2022-06-21T22:14:11', '2022-06-21T22:14:41', '2022-06-21T22:15:11', '2022-06-21T22:15:41', '2022-06-21T22:16:11', '2022-06-21T22:16:41', '2022-06-21T22:17:11', '2022-06-21T22:17:41', '2022-06-21T22:18:11', '2022-06-21T22:18:41', '2022-06-21T22:19:11', '2022-06-21T22:19:41', '2022-06-21T22:20:11', '2022-06-21T22:20:41', '2022-06-21T22:21:11', '2022-06-21T22:21:41', '2022-06-21T22:22:11', '2022-06-21T22:22:41', '2022-06-21T22:23:11', '2022-06-21T22:23:41', '2022-06-21T22:24:11', '2022-06-21T22:24:41', '2022-06-21T22:25:11', '2022-06-21T22:25:41', '2022-06-21T22:26:11', '2022-06-21T22:26:41', '2022-06-21T22:27:11', '2022-06-21T22:27:41', '2022-06-21T22:28:11', '2022-06-21T22:28:41', '2022-06-21T22:29:11', '2022-06-21T22:29:41', '2022-06-21T22:30:11', '2022-06-21T22:30:41', '2022-06-21T22:31:11', '2022-06-21T22:31:41', '2022-06-21T22:32:11', '2022-06-21T22:32:41', '2022-06-21T22:33:11', '2022-06-21T22:33:41', '2022-0